Bro I never worked with python, so i did not know there was a good tool for writing notes like this IPYNB, will do most of the thing with this now yay
Credits to [Zigtur](https://github.com/zigtur/Groth16/blob/main/Groth16.ipynb) i was reading his notes for this part got to know about this worderful tool


Okay Now lets get started

# Groth16
Groth16 is a zero knowledge based algorithm in which a prover (may or may not be malicious) proves a statement's soundness.
In this jupiternote book, we are going to implement a very simple Groth16 to prove that our problem statement is true.
This is complete mirror of Zigtur's notes However, am gonna take a different circuit eqaution than him


# Context and problem
In the below sections we will implement and use groth16 to prove our statement without leaking any other informations

In Zigtur's notes he decided to take the the eqaution $x^3+3x^2y+2z^2y^3=169695$

I will take the following eqaution instead
$$
x^3 + 2xy + z^2 = 73
$$
this seems pretty fine i would say.


# Problem's solution
The prover needs to know the vlaues x,y,z. In Our example those values are:
x=3,y=7,z=2

These values will remain private, they will never be sent to the verifier.

## Initialize Python Setup
In our implementation, we will be working on BN128 elliptic curve, just like Zigtur did. As such we will use a finite field which is the number of points of this curve (order of the curve). 
In python we need to import the galios library for finite fields. For EC, we need to have py_ecc.numpy will be used to manipulate matrices



In [4]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize, curve_order, final_exponentiate
import galois

print("Initializing a large field...")
GF = galois.GF(curve_order)
print("Field initialized")

Initializing a large field...
Field initialized


## Arithmetic circuit
Our equation out= $$x^3 + 2xy + z^2 = 73$$

can be broken down into an arithmetic circuit.
From this arithmetic circuit, we define a set of constraints which has the form O=L * R 
$$v_1= x*x$$

$$v_2=x*v_1$$

$$t=x*y$$

$$v_3=2t$$
$$v4=z*z$$
$$(v_2 + v_3 +v_4).1=out$$
$$out = v_2 +v_3+v_4$$


# Rank 1 constraint System
Now that we have our arithmetic circuit, we can turn it into a rrank1 constraint system, we need to adapt out set of constraints to easily match the required R1Cs format
To have a proper $O=L*R$ form, we can create 4 matrices. the first matric is named a. which is the mitness matric. the value of the S matric is know by the prover.

$$w = [1, x, y, z, v1, v2, t, v3, v4, out]$$

$$w = [1, 3, 7, 2, 9, 27, 21, 42, 4, 73]$$
Then, we can produce 3 matrices that will represent the constrainst. It sort of represents the wy the s element are interconnected

$$
L =
\begin{bmatrix}
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
-73 & 0 & 0 & 0 & 0 & 1 & 2 & 1
\end{bmatrix}
$$

$$
R =
\begin{bmatrix}
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$

$$
O =
\begin{bmatrix}
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$
The final R1CS representation if Ra # La =Os each column of matrices L R O corresponds to an element of S. Each row of matrices L R O corresponds to one of the constraits of the arithmetic circuit.



In [6]:
import numpy as np
# variable order:
# [1, x, y, z, v1, v2, t, v3, v4, out]

L = [
    # (1) x * x = v1
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],

    # (2) v1 * x = v2
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],

    # (3) x * y = t
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],

    # (4) (2t) * 1 = v3
    [0, 0, 0, 0, 0, 0, 2, 0, 0, 0],

    # (5) z * z = v4
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],

    # (6) (v2 + v3 + v4) * 1 = out
    [0, 0, 0, 0, 0, 1, 0, 1, 1, 0],
]


R = [
    # x
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],

    # x
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],

    # y
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],

    # 1
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],

    # z
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],

    # 1
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]


O = [
    # v1
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],

    # v2
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],

    # t
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],

    # v3
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],

    # v4
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],

    # out
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
]
import numpy as np
import galois

GF = galois.GF(97)  # or BN254 scalar field

# convert matrices into GF arrays
L = GF(np.array(L))
R = GF(np.array(R))
O = GF(np.array(O))

s = GF([1, 3, 7, 2, 9, 27, 21, 42, 4, 73])

result = (O @ s) == (L @ s) * (R @ s)

print(result.all())



True


# QAP

aight, we have finished One part of the this project now comes the part, in which we will convert that R1CS into a QAP. less gooo
This transformation consists of turning our constraint matrices into polynomials. The main goal of this transformation is succinctness. Thanks to the Schwartz-Zippel Lemma, we know that evaluating two polynomials at a random point x allows nearly be certain that they are the same polynomial if they return the same value y.

Evaluation is way more succint that matrices evaluation.

To achieve succinct, we will derivate the a QAP from our R1CS. Our goal is to get u*v=w if Ls # Rs =Os the Transformation is made possible thanks to a ring homomorphism between matrices and polynomials

$$x= [1,2,3,4,5,6,7]$$


# R1CS to QAP
For each column of each matrix, we will interpolate a polynomial. The interpolation will be done usingg lagrange interpolation. the interpolated polynomial will encode all the values of the column

for example, the third cloum of L is:
$$
\begin{bmatrix}
0 \\
0 \\
0 \\
0 \\
0 \\
\end{bmatrix}
$$
so the interpolated $$P_{L_3}(1)=0$$
$$P_{L_3}(2)=0$$
$$P_{L_3}(3)=0$$
$$P_{L_3}(4)=0$$
$$P_{L_3}(5)=00$$



In [ ]:
x= GF(np.array([1,2,3,4,5,]))
L3_column= GF(np.array([0,0,0,0,0]))
L3_poly= galois.lagrange_poly(x,L3_column)

print("The resulting polynomial is:",L3_poly)
print("L3_poly(1) == 1:", L3_poly(1) == 0)
print("L3_poly(2) == 1:", L3_poly(2) == 0)
print("L3_poly(3) == 0:", L3_poly(3) == 0)
print("L3_poly(4) == 0:", L3_poly(4) == 0)
print("L3_poly(5) == 0:", L3_poly(5) == 0)


The resulting polynomial is: 12x^4 + 42x^3 + 78x^2 + 68x + 92
L3_poly(1) == 1: False
L3_poly(2) == 1: False
L3_poly(3) == 0: True
L3_poly(4) == 0: True
L3_poly(5) == 0: True


Now that we have achieve it for one column, we can do this process to get the interpolated polynomials for all columsn of each matrix.
The following python code does it!


In [11]:
def interpolate_column(col, nb):
    xs = GF(np.arange(1,nb+1))
    return galois.lagrange_poly(xs, col)

def get_polys_of_matrix(matrix):
    polys = []
    nb_of_rows = len(matrix)
    nb_of_columns = len(matrix[0])
    # for each column
    for col_id in range(nb_of_columns):
        column = []
        for row in range(nb_of_rows):
            column.append(matrix[row][col_id])
        polys.append(interpolate_column(GF(np.array(column)), nb_of_rows))
    return np.array(polys)
    

## computes all interpolated polynomials for L, R, and O
U_polys = get_polys_of_matrix(L)
V_polys = get_polys_of_matrix(R)
W_polys = get_polys_of_matrix(O)

def get_polys_of_S_matrix(matrix):
    polys = []
    nb_of_columns = len(matrix)
    # for each column
    for col_id in range(nb_of_columns):
        polys.append(galois.Poly([matrix[0]], field=GF))
    return np.array(polys)
    
A_polys = get_polys_of_S_matrix(s)

# Summing all the polynamials of the collection into one polynomial
U = galois.Poly([0], field=GF)
V = galois.Poly([0], field=GF)
W = galois.Poly([0], field=GF)
for i in range(len(U_polys)):
    U += U_polys[i]
    V += V_polys[i]
    W += W_polys[i]

In [12]:
Ua = galois.Poly([0], field=GF)
for i in range(len(s)):
    Ua += U_polys[i] * s[i]

Va = galois.Poly([0], field=GF)
for i in range(len(s)):
    Va += V_polys[i] * s[i]

Wa = galois.Poly([0], field=GF)
for i in range(len(s)):
    Wa += W_polys[i] * s[i]